Импортируем необходимые библиотеки, а также напишем пару функций для обработки данных:
- tf_binarize: фактически LabelEncoder, ставит 1 вместо `t`, 0 вместо `t`;
- quantile_bucketing: группирует хвост распределения заданного веса;
- exist_binarize: ставит 1, если столбец содержит данные, 0 иначе;
- date_transform: переводит время в количество секунд с начала отсчета;

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.pipeline import make_pipeline, Pipeline


# Handmade transformers
def tf_binarize(col):
    return np.where(col == 't', 1, 0)

def quantile_bucketing(col, level = 0.95):
    q = col.quantile(level)
    return np.where(col > q, q, col)

def exist_binarize(col):
    return np.where(col.isna(), 0, 1)

def date_transform(col):
    return list(map(lambda x: (x - datetime.datetime(1970, 1, 1)).total_seconds(), col))


# custom metric
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Для начала считаем и объединим все данные, чтобы подобрать оценить разбросы фичей.

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df = pd.concat([train, test])
del train, test
df.shape

(74815, 43)

## 1. Extra data/features

## 1.1 Calendar

Из календаря извлечем количество дней, в которые был доступен данный лот.

In [3]:
calendar = pd.read_csv('calendar.csv')
calendar.available = tf_binarize(calendar.available)
calendar.rename(columns = {'listing_id' : 'id'}, inplace=True)

In [4]:
current_date = '2019-11-04' 
year_ago_date = '2018-11-04'

av_year = calendar[calendar.date > year_ago_date].groupby('id')['available'].sum()
av_year = av_year.reset_index()

av_year.rename(columns = {'available' : 'available_year'}, inplace=True)
del calendar

## 1.2 Reviews

Из отзывов возьмем количество уникальных отзывов о лоте за все время.

In [5]:
reviews = pd.read_csv('reviews.csv')
reviews = reviews[~reviews.comments.isna()]

unique_comments = reviews.groupby('listing_id')['reviewer_id'].nunique().reset_index()
unique_comments.rename(columns = {'reviewer_id' : 'unique_reviewers', 'listing_id' : 'id'}, inplace=True)
del reviews

## 1.3 Host owns

Данная фича показывает, сколько объектов владельца лота выставлено для аренды.

In [6]:
host_owns = df.groupby('host_id')['id'].count().reset_index()
host_owns.rename(columns = {'id' : 'host_realty_count'}, inplace=True)
#host_owns.host_realty_count = quantile_bucketing(host_owns.host_realty_count, 0.99)

## 2. Feature Engineering

Для удобства все фичи разбиты по спискам, с указанием последующего преобразования для них.

In [7]:
target = ['price']

looks_fine_features = [
    'latitude',
    'longitude',
    'bathrooms',
    'minimum_nights',
    'host_since',
]

drop_features = [
    'id', # no way
    'name', # looks noisy
    'summary', 'space', 'description', # aren't they always positive?
    'access', 'interaction', 'house_rules', # lot of empties and hard to use
    'zipcode', # too much different values
    'bed_type', # noisy and unbalanced
    'square_feet', # too many n/a
    'experiences_offered', #3 not none
    # 'host_response_rate', # -> [0, 1]
    'neighbourhood_cleansed', # OHE
    'property_type', # OHE + bucketing (b)
    'extra_people', #wtf distrib
    'host_id', # used
    'amenities', # used
]

tf_feats = [
    'host_is_superhost', # tf
    'host_identity_verified', # tf
    'is_location_exact', # tf
    'require_guest_phone_verification', # DISBALANCED
    'require_guest_profile_picture', # DISBALANCED
    'host_has_profile_pic', # DISBALANCED
]

tf_is_exist_feats = [
    'host_about',
    'neighborhood_overview',
    'notes',
    'transit',
]

to_bucket_feats = [
    'bathrooms', # b
    'bedrooms', # b
    'beds', # ...
    'guests_included',
]

to_dummy_feats = [
    'room_type',
    'cancellation_policy',
    'host_response_time', # OR binarize?
]

fill_with_median = [
    'security_deposit',
    'cleaning_fee',
]

Далее идет самописный класс для преобразования признаков. Изначально он скармливался в pipeline, но поскольку test известен, впоследствии преобразование применялось ко всему датасету.

In [8]:
class RealtyDataTransform:
    def fit(self, X, y):
        return self

    def transform(self, df):
        # делаем encoding для dummy признаков
        df = pd.get_dummies(df, columns=to_dummy_feats)

        # Бинаризуем 
        for col in tf_feats:
            df[col] = tf_binarize(df[col])

        # Бинаризуем по принципу пустое поле или нет
        for col in tf_is_exist_feats:
            df[col] = exist_binarize(df[col])

        # преобразуем дату, чтобы затем отскалировать ее 
        # (по сути, признак отвечает за длительность нахождения пользователя на сайте)
        # если данных нет, считаем что пользователь новый
        df.host_since.astype('datetime64[ns]')
        df.host_since = date_transform(df.host_since.astype('datetime64[ns]'))
        df.host_since.fillna(df.host_since.max(), inplace=True)

        # дополняем данными из других источников
        df = pd.merge(df, av_year, on = 'id', how='left')
        df = pd.merge(df, unique_comments, on = 'id', how='left')
        df = pd.merge(df, host_owns, on = 'host_id', how='left')

        # https://www.kaggle.com/brittabettendorf/predicting-prices-xgboost-feature-engineering
        # здесь взял идею, как преобразовать столбец `amenities`
        df['Laptop_friendly_workspace'] = df['amenities'].str.contains('Laptop friendly workspace').astype(np.int)
        df['TV'] = df['amenities'].str.contains('TV').astype(np.int)
        df['Family_kid_friendly'] = df['amenities'].str.contains('Family/kid friendly').astype(np.int)
        df['Host_greets_you'] = df['amenities'].str.contains('Host greets you').astype(np.int)
        df['Smoking_allowed'] = df['amenities'].str.contains('Smoking allowed').astype(np.int)

        # Заполним нулями пропуски признака host_response_rate и переведем проценты в дроби
        df.host_response_rate.replace(np.NaN, '0', inplace=True)
        df.host_response_rate = df.host_response_rate.apply(lambda x: int(x.replace('%','')) / 100)

        # убираем то что не требуется для модели
        df.drop(drop_features, axis=1, inplace=True)

        # заполняем медианой соотв. столбцы
        df[fill_with_median] = df[fill_with_median].apply(pd.to_numeric, errors='coerce')
        df[fill_with_median] = df[fill_with_median].fillna(df.median())

        # все что осталось пустым, заполним минимумом по признаку
        df.fillna(df.min(), inplace=True)
        return df

## 3. Build datasets

In [9]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

# Уберем выбросы из трейна
train = train[train.price > 0]
# train = train[train.price < 700] (работало в худшую сторону)
y = train.price
train.drop(['price'], axis=1, inplace=True)

Преобразуем данные. Теперь X -набор тренировочных данных.

In [10]:
fe = RealtyDataTransform()
scaler = MinMaxScaler()
# pipe = Pipeline(steps=[('feature_engineering', fe), ('scaler' , scaler), ('model', linear_model.LinearRegression())])

scaler.fit(fe.transform(df.drop(['price'], axis=1)))
X = scaler.transform(fe.transform(train))
test = scaler.transform(fe.transform(test))
del train, df

## 4. Modeling

Выделим валидационную выборку.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y.to_numpy(), test_size=0.25, random_state=42)

Будем использовать градиентный бустинг пакета LightGBM.

In [12]:
 import lightgbm as lgbm
 params = {
     'boosting_type': 'gbdt',
     'objective': 'mape',
     'num_leaves' : 1023,
     'learning_rate' : 0.01,
     'n_estimators': 2500,
     'bagging_fraction' : 0.7,
     'feature_fraction' : 0.7,
 }
 lgb = lgbm.LGBMRegressor(**params)

Посмотрим качество модели обученной на X_train.

In [13]:
lgb.fit(X_train, np.log(y_train))
y_pred_lgb = lgb.predict(X_test)
mean_absolute_percentage_error(y_test, np.exp(y_pred_lgb))

22.296522562361734

Улучшим модель средствами кросс-валидации.

In [14]:
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits)

model = lgbm.LGBMRegressor(**params)

valid_ans = np.zeros(y_test.shape[0])
test_ans = np.zeros(test.shape[0])
current_fold = 0
for train_index, test_index in kf.split(X_train):
    current_fold += 1
    X_train_cv, X_test_cv = X_train[train_index], X_train[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    model.fit(X_train_cv, np.log(y_train_cv))
    y_pred_fold = model.predict(X_test_cv)
    print("Fold MAPE: ", mean_absolute_percentage_error(y_test_cv, np.exp(y_pred_fold)))
    print("Fold #", current_fold, "has processed.")
    valid_ans += model.predict(X_test)
    test_ans += model.predict(test)
print("Valid prediction: ", mean_absolute_percentage_error(y_test, np.exp(valid_ans / n_splits)))
# формируем итоговое предсказание
test_preds = np.exp(test_ans / n_splits)

Fold MAPE:  23.07836020837106
Fold # 1 has processed.
Fold MAPE:  23.024808256310124
Fold # 2 has processed.
Fold MAPE:  23.914602240496702
Fold # 3 has processed.
Fold MAPE:  22.71684207629346
Fold # 4 has processed.
Fold MAPE:  23.61185078798279
Fold # 5 has processed.
Valid prediction:  22.41377424620599


## 5. Create a submission

Собираем итоговый сабмит.

In [15]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['price'] = test_preds
sub_df.head()

,id,price
0,9554,36.398165
1,11076,52.375804
2,13913,46.244453
3,17402,249.368863
4,24328,115.143515


In [16]:
sub_df.to_csv('submission.csv',index=None)

## 6. Conclusion

Итоговый скор на паблике: 23,56 (strong baseline - 24,63)

Итоговый скор на прайвате: 

Что не сработало: 
- во многих работах по похожим датасетам хорошо работало извлечение размера помещения (с предсказанием пропусков, например, линейной моделью), однако почечему-то у меня это не дало никакого прироста;
- использование расстояния до центра города также не дало никакого преимущества над обычными координатами;
- другие фичи по данным из Calendar, Reviews (меньшие временнЫе окна, например);

На что не хватило времени:
- фактически всё NLP (например, классификация отзывов по удовлетворенности сторонней моделью, затем составление рейтинга лота);
- другие приложения геоданных (ближайшие ТЦ, исторические объекты, удаленность от города).

Спасибо за внимание!